In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Leer IDs desde un archivo de texto
file_path = r'C:\Users\Laura\Desktop\Henry_Automatizaciones\ids.txt'
with open(file_path, 'r') as file:
    id_list = [line.strip() for line in file if line.strip().isdigit()]

# Lista para almacenar información de trabajos
job_list = []

# Obtener información de cada trabajo
for job_id in id_list:
    # URL de la vacante
    job_url_api = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    job_url_decent = f"https://www.linkedin.com/jobs/view/{job_id}"  # URL limpia

    # Solicitar datos del trabajo
    job_response = requests.get(job_url_api)
    if job_response.status_code != 200:
        print(f"Error al obtener datos para el ID: {job_id}")
        continue  # Saltar si hay error en la solicitud

    # Parsear datos del trabajo
    job_soup = BeautifulSoup(job_response.text, "html.parser")
    job_post = {
        "Estado": "Postulado",
        "Posición": None,
        "Plataforma": "LinkedIn",
        "Empresa": None,
        "Rubro": None,
        "Nombre del Reclutador": None,
        "Link Posición": job_url_decent,
        "Link Empresa": None,
        "Link Reclutador": None,
        "Publicación": None,
        "Comentarios": None,
    }

    # Extraer detalles del trabajo
    try:
        job_post["Posición"] = job_soup.find("h2", {"class": "topcard__title"}).text.strip()
    except AttributeError:
        pass

    try:
        company_tag = job_soup.find("a", {"class": "topcard__org-name-link"})
        job_post["Empresa"] = company_tag.text.strip()
        job_post["Link Empresa"] = company_tag.get("href")
    except AttributeError:
        pass

    try:
        criterios = job_soup.find_all("span", {"class": "description__job-criteria-text"})
        if len(criterios) > 0:
            job_post["Rubro"] = criterios[3].text.strip()
    except AttributeError:
        pass

    try:
        recruiter_tag = job_soup.find("a", {"class": "base-card__full-link"})
        job_post["Nombre del Reclutador"] = recruiter_tag.text.strip()
        job_post["Link Reclutador"] = recruiter_tag.get("href")
    except AttributeError:
        pass

    # Extraer información de la publicación
    try:
        publication_time = job_soup.find("span", {"class": "posted-time-ago__text topcard__flavor--metadata"})
        job_post["Publicación"] = publication_time.text.strip() if publication_time else "No disponible"
    except AttributeError:
        job_post["Publicación"] = "No disponible"

    # Agregar el trabajo a la lista
    job_list.append(job_post)

# Guardar los resultados en un archivo Excel
jobs_df = pd.DataFrame(job_list)
jobs_df.to_excel("LinkedIn_Jobs_Info.xlsx", index=False, engine="openpyxl")

print("Se han guardado los resultados en 'ids_LinkedIn_Jobs_Info.xlsx'.")


Se han guardado los resultados en 'ids_LinkedIn_Jobs_Info.xlsx'.


In [3]:
# Create a pandas DataFrame using the list of job dictionaries 'job_list'
jobs_df = pd.DataFrame(job_list)
jobs_df

,Estado,Posición,Plataforma,Empresa,Rubro,Nombre del Reclutador,Link Posición,Link Empresa,Link Reclutador,Publicación,Comentarios
0,Postulado,Junior Data Analyst,LinkedIn,NielsenIQ,Market Research,None,https://www.linkedin.com/jobs/view/4128406753,https://www.linkedin.com/company/nielseniq?trk...,None,No disponible,None
